# Download data from the NOAA Tides and Currents site

In [1]:
import os
import requests

## Describe data to download

In [2]:
product    = 'hourly_height'
begin_year = 1983
end_year   = 2015
datum      = 'NAVD'
station    = '8594900'
time_zone  = 'gmt'
units      = 'english'
formatt    = 'csv'
string = "http://tidesandcurrents.noaa.gov/api/datagetter?product={}&application=web_services&begin_date={}&end_date={}&datum={}&station={}&time_zone={}&units={}&format={}"

## Define download function

In [3]:
def download_file(url,filename):
    local_filename = filename
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

## CD to the directory where the data will be stored

In [5]:
os.chdir(r"C:\Users\jdorvinen\Documents\GitHub\hydro-notebooks\tides\data")

## Download files

In [7]:
filenames = []
for i in range((end_year-begin_year+1)):
    year = begin_year+i
    filename   = station+'_'+product+'_'+str(year)+'.csv'
    filenames.append(filename)
    begin_date = '{}0101'.format(str(year))
    end_date   = '{}1231'.format(str(year))
    url        = string.format(product,begin_date,end_date,datum,station,time_zone,units,formatt)
    download_file(url,filename)

## Combine files

In [8]:
combinedfiles = station+'_'+product+'_combined.txt'
with open(combinedfiles, 'w') as outfile:
    with open(filenames[0],'r') as r:
        header = r.readline()
        outfile.write(header)
    r.close()
    for fname in filenames:    
        with open(fname) as infile:
            for line in infile:
                if line != header:
                    outfile.write(line)